
# Robo-Walk Simulation)

### Code Adjustment from Roberto Sannazzaro, [‘How to Automatize a Warehouse Robot’](https://medium.datadriveninvestor.com/get-started-with-q-learning-with-python-how-to-automatize-a-warehouse-robot-7bfae0180301)

## 地圖設計：
<img src="https://i.ibb.co/LrTfrgc/warehouse.png">

In [15]:
import numpy as np

## environment setup

In [16]:
# Poisition-Code
location_to_state = {         
                     'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6,
                     'H': 7,
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11}

## Actions

In [17]:
actions = [0,1,2,3,4,5,6,7,8,9,10,11]

## 定義行動限制，假設G點有最高優先度, 故獎勵 = 1000

In [18]:
# 定義 1: 可到，0:不可到
R = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1000, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

## 依TD(1) 演算法更新行動值函数

In [19]:
# Parameter
gamma = 0.75
alpha = 0.9

# 行動值函數初始值為 0
Q = np.array(np.zeros([12,12]))

# 1000 epoch
for i in range(1000):
    # Random Start Position
    current_state = np.random.randint(0,12)
    playable_actions = []
    for j in range(12):
        if R[current_state, j] > 0:
            playable_actions.append(j)
    # Random Action
    next_state = np.random.choice(playable_actions)
    # 更新行動值函数
    TD = R[current_state, next_state] + gamma*Q[next_state, \
            np.argmax(Q[next_state,])] - Q[current_state, next_state]
    Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD

## Update Result：越靠近G，行動函数數值越高

In [20]:
import pandas as pd

q_values = pd.DataFrame(Q, columns=[location for location in location_to_state])
s = q_values.round().style.background_gradient(cmap='GnBu')
s

,A,B,C,D,E,F,G,H,I,J,K,L
0,0.000000,1686.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1264.000000,0.000000,2247.000000,0.000000,0.000000,1264.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,1686.000000,0.000000,0.000000,0.000000,0.000000,2994.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2247.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,713.000000,0.000000,0.000000,0.000000
5,0.000000,1686.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,949.000000,0.000000,0.000000
6,0.000000,0.000000,2245.000000,0.000000,0.000000,0.000000,3991.000000,2246.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,1686.000000,0.000000,0.000000,2994.000000,0.000000,0.000000,0.000000,0.000000,1685.000000
8,0.000000,0.000000,0.000000,0.000000,535.000000,0.000000,0.000000,0.000000,0.000000,949.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,1265.000000,0.000000,0.000000,712.000000,0.000000,1260.000000,0.000000


## 重新定義行動限制

In [21]:
R = np.array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
              [1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
              [0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0],
              [0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1],
              [0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
              [0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0],
              [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1],
              [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]])

## 代碼與位子對照表

In [22]:
state_to_location = {state: location for location, 
                     state in location_to_state.items()}
state_to_location            

{0: 'A',
 1: 'B',
 2: 'C',
 3: 'D',
 4: 'E',
 5: 'F',
 6: 'G',
 7: 'H',
 8: 'I',
 9: 'J',
 10: 'K',
 11: 'L'}

## 定義路徑訓練函数

In [23]:
def route(starting_location, ending_location):
    # starting_location, ending_location
    # 位置轉為代碼
    ending_state = location_to_state[ending_location]
    # 终點有最高優先度
    R_new = np.copy(R)
    R_new[ending_state, ending_state] = 1000
    
    # 策略评估：1000 epoch
    Q = np.array(np.zeros([12,12]))
    for i in range(1000):
        current_state = np.random.randint(0,12)
        playable_actions = []
        for j in range(12):
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        # 任意行動
        next_state = np.random.choice(playable_actions)
        # 更新行動值函数
        TD = R_new[current_state, next_state] + gamma * \
            Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        Q[current_state, next_state] = Q[current_state, next_state] + alpha * TD
        
    # 策略改善：依 TD 找尋最佳路徑
    route = [starting_location]
    next_location = starting_location
    while (next_location != ending_location):
        starting_state = location_to_state[starting_location]
        next_state = np.argmax(Q[starting_state,])
        next_location = state_to_location[next_state]
        route.append(next_location)
        starting_location = next_location
    return route

## Test E --> G 最佳路

In [24]:
route('E', 'G')

['E', 'I', 'J', 'F', 'B', 'C', 'G']

In [25]:
# Test A --> K 最佳路
route('A', 'K')

['A', 'B', 'F', 'J', 'K']

In [26]:
# 3 個點的路
def best_route(starting_location, intermediary_location, \
               ending_location):
    # 3 個點的路 = 2 個點的路 + 2 個點的路
    return route(starting_location, intermediary_location) + \
            route(intermediary_location, ending_location)[1:]

## 测试 E --> K --> G 最佳路由

In [27]:
best_route('E', 'K', 'G')

['E', 'I', 'J', 'K', 'L', 'H', 'G']

In [28]:
# 测试 A --> G --> K 最佳路由
initial = "A" 
intermediary = "G" 
final = "K" 
best = best_route(initial, intermediary, final)
print('最佳路: ')
print(*best, sep=', ')

最佳路: 
A, B, C, G, H, L, K


## References 

* [An introduction to Q-Learning](https://www.freecodecamp.org/news/an-introduction-to-q-learning-reinforcement-learning-14ac0b4493cc/)
* [Reinforcement learning](https://medium.com/machine-learning-for-humans/reinforcement-learning-6eacf258b265)
*[Math of Q-Learning](https://medium.com/datadriveninvestor/math-of-q-learning-python-code-5dcbdc49b6f6)